<a href="https://colab.research.google.com/github/gbardaklis/Mini-Project-1/blob/main/Q3Embeddings_as_Features_Wiki_GigaWord_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Needed Imports
import gensim.downloader as api
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from sklearn import metrics

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# More imports
import json
import pandas as pd
import numpy as np
import warnings


In [ ]:
with open("/content/goemotions.json") as f:
    # data2 = json.loads(f.read())
    data = json.load(f)

# importing the json file into a dataframe 
df = pd.DataFrame(data, columns = ['Post', 'Emotion', 'Sentiment'])
postsDict = df['Post'].values.tolist()
# postsDict

In [ ]:
emotionDict = df['Emotion'].values.tolist()
# emotionDict
sentimentDict = df['Sentiment'].values.tolist()
# sentimentDict

In [ ]:
# Displays a List of Pretrained Models avail on gensim
print(list(api.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [ ]:
# 3.1 
corpusNews = api.load('glove-wiki-gigaword-300')

[==================================================] 100.0% 376.1/376.1MB downloaded


In [ ]:
# 3.2
tokenCount = 0
tokenPost = []

for x in postsDict:
  tokens = word_tokenize(x)
  tokenPost.append(tokens)
  tokenCount = tokenCount + len(tokens)
print(f'The total number of tokens is {tokenCount}')

The total number of tokens is 2642128


In [ ]:
# fail counter for punctuation and words not found in corpus 
failed = 0
averageEmb = []
averageEmbAll = []

# Take the X TokenPost
for index, iterator in enumerate(tokenPost):
  for x in iterator:
    try:
      # Assign Corpus[x] to single word
      singleWord = corpusNews[x]
      # New List append that word
      averageEmb.append(singleWord)
    except KeyError:
      # Doesn't fnd throw error increment counter
      failed += 1
  if len(averageEmb) != 0:
    average = np.average(averageEmb, axis = 0)
    averageEmbAll.append(average)
    averageEmb.clear()
  else: 
    averageEmbAll.append([0] * 300)
# 3.3
print(f'The failed count is: {failed}')

# 3.4 
hitRate = ((tokenCount - failed) / tokenCount) *100
print(f'The hit rate is as follows: {hitRate}')

The failed count is: 387248
The hit rate is as follows: 85.34332931636924


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# splitting data 80% train 20% test
post_train, post_test, emotion_train, emotion_test = train_test_split(averageEmbAll, emotionDict, test_size = 0.2)

In [ ]:
# splitting data 80% train 20% test
post_train_sent, post_test_sent, sentiment_train, sentiment_test = train_test_split(averageEmbAll, sentimentDict, test_size = 0.2)

In [ ]:
# 3.5 Base-MLP with the default parameters
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(early_stopping=True)
%time mlp.fit(post_train, emotion_train)
%time mlp.fit(post_train, sentiment_train)

CPU times: user 56.9 s, sys: 22.8 s, total: 1min 19s
Wall time: 40.8 s
CPU times: user 1min 18s, sys: 31.8 s, total: 1min 50s
Wall time: 57.2 s


MLPClassifier(early_stopping=True)

In [ ]:
# 3.7
# Open Text Word doc for Writing to
performanceTxt = open("performanceWordEmb_Wiki_GigaWord.txt", "w")
# Write to Text Word Doc the Accuracy and the Classification Report for BASE 
performanceTxt.write(f" The following is the information for the Base MLP Classifier \n  " )
performanceTxt.write(f" This is the Accuracy of the Tests for the emotion model \n {mlp.score(post_test, emotion_test) * 100} \n  " )
performanceTxt.write(f" This is the Accuracy of the Tests for the sentiment model \n {mlp.score(post_test, sentiment_test) * 100} \n  " )
emotion_pred = mlp.fit(post_train, emotion_train).predict(post_test)
performanceTxt.write(f" This is the Classification Report for the emotion model \n {metrics.classification_report(emotion_test, emotion_pred,labels=np.unique(emotion_test), zero_division=0)} \n  " )
sentiment_pred = mlp.fit(post_train, sentiment_train).predict(post_test)
performanceTxt.write(f" This is the Classification Report for the sentiment model \n {metrics.classification_report(sentiment_test, sentiment_pred,labels=np.unique(sentiment_test), zero_division=0)} \n  " )
print("Wrote everything to the file good job!")

Wrote everything to the file good job!
